<table><tbody><tr><th><p><img alt="Emblema" src="https://cdn6.aptoide.com/imgs/6/f/4/6f4821daa840da8fe971445350759fe5_icon.png" style="width:150px;"></p></th><th><p><strong>Inteligencia Artificial</strong></p><p><strong>Grado en Ingeniería Informática en Sistemas de Información – Curso 2024/2025</strong></p><p><strong>ENSEÑANZAS PRÁCTICAS Y DE DESARROLLO</strong></p><h1>EPD 6: Machine Learning – Sistemas de recomendación</h1></th></tr></tbody></table>

____

## Objetivos
- Implementación en Python de un algoritmo de sistemas de recomendación.

___

## Bibliografía Básica
- Recommender systems. Charu C. Aggarwal. Springer, 2016. Disponible online: http://pzs.dstu.dp.ua/DataMining/recom/bibl/1aggarwal_c_c_recommender_systems_the_textbook.pdf

- Recommender systems handbook. Francesco Ricci, Lior Rokach, Bracha Shapira, Paul B. Kantor. Springer, 2011. Disponible online: https://www.cse.iitk.ac.in/users/nsrivast/HCC/Recommender_systems_handbook.pdf

___

In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.optimize as opt

## Ejercicios
Implementar un algoritmo que recomiende películas a los usuarios. Para ello, usar el fichero “ex8_movies.mat” que contiene datos de películas clasificadas por los usuarios en una escala del 1 al 5. En concreto, 943 usuarios han clasificado 1682 películas. Las películas se identifican con 10 características relativas a su contenido. El objetivo del algoritmo es predecir la puntuación que le daría un usuario a una película que no ha visto aún y recomendar a ese usuario las películas con las puntuaciones más altas.

#### EJ01. 

Cargar el dataset y prepararlo para el algoritmo usando 2 matrices. La matriz Y almacenará las clasificaciones de las películas y la matriz R contendrá solamente valores binarios donde R(i,j) = 1 significará que el usuario j clasificó la película i y R(i,j) = 0 indicará que no la clasificó. Ambas matrices tendrán como dimensión: número de películas x número de usuarios. La media de las puntuaciones que recibe la primera película (Toy Story) debe ser aproximadamente 3.878319. Almacenar en las matrices de parámetros X y Theta los valores pre-entrenados disponibles en el fichero “ex8_movieParams.mat”. Las dimensiones de X deben ser número de películas x número de características y las de Theta número de características x número de usuarios. Compruebe las dimensiones y actúe en caso de que no coincidan.

##### Solución:

In [2]:
# =============== EJ1: Cargar datos ================
print('Loading movie ratings dataset.')
movies = sio.loadmat("ex8_movies.mat")
Y = movies['Y'] # [n_items, n_users] puntuaciones de 1-5
R = movies['R'] # [n_items, n_users] R(i,j)=1 si usuario j puntuó pelicula i
print("Shape de Y: ", Y.shape)  # [n_items, features]
print("Shape de R: ", R.shape)  # [n_items, features]

print('\tAverage rating for the first movie (Toy Story): ', Y[0, np.where(R[0, :] == 1)[0]].mean(), "/5\n")

#  Cargar parámetros preentrenados (X, Theta, num_users, num_movies, num_features)
    
params_data = sio.loadmat('ex8_movieParams.mat')
X = params_data['X']
Theta = params_data['Theta'] 
#...
print("Shape de X: ", X.shape)  # [n_items, features]
print("Shape de Theta: ", Theta.shape)  # [features, n_users]


Loading movie ratings dataset.
Shape de Y:  (1682, 943)
Shape de R:  (1682, 943)
	Average rating for the first movie (Toy Story):  3.8783185840707963 /5

Shape de X:  (1682, 10)
Shape de Theta:  (943, 10)


#### EJ02.
Implementar la función coste sin regularización para un sistema de recomendación de filtrado colaborativo en cofiCostFuncSinReg siguiendo la fórmula indicada en EB. El coste se acumula para el usuario j y la película i sólo si R(i,j)= 1. Si usa las matrices de parámetros X y Theta almacenadas en el fichero para los 4 primeros usuarios, 5 primeras películas y 3 primeros atributos/características, el coste debe ser 22.22 aproximadamente.

##### Solución:

In [3]:
def cofiCostFuncSinReg(params, Y, R, num_features):
    num_movies, num_users = Y.shape
    
    # Desenrollar parámetros
    X = np.reshape(params[:num_movies*num_features], (num_movies, num_features), 'F')
    Theta = np.reshape(params[num_movies*num_features:], (num_features, num_users), 'F')
    J = 0
    
    error = np.multiply(np.dot(X, Theta) - Y, R)
    squared_error = np.power(error, 2)
    J = 1/2 * np.sum(squared_error)
    return J

In [4]:
### Subconjunto de datos para que ejecute más rápidamente
users = 4
movies = 5
features = 3
X_sub = X[:movies, :features]
Theta_sub = Theta[:features, :users]
Y_sub = Y[:movies, :users]
R_sub = R[:movies, :users]
params =  np.hstack((np.ravel(X_sub, order="F"), np.ravel(Theta_sub, order="F"))) # Desenrollar: primero X_sub luego Theta_sub
J = cofiCostFuncSinReg(params, Y_sub, R_sub, features)
print("Cost without regularization at loaded parameters: ", J, "(this value should be about 22.22)")


Cost without regularization at loaded parameters:  57.356479775998096 (this value should be about 22.22)


#### EJ03.
Implementar la función gradiente sin regularización en cofiGradientFuncSinReg. Usar la función auxiliar checkNNGradientsSinReg.py para verificar que los gradientes están bien calculados.

##### Solución:

In [5]:
def cofiGradientFuncSinReg(params, Y, R, num_features):
    num_movies, num_users = Y.shape
    
    # Desenrollar parámetros
    X = np.reshape(params[:num_movies*num_features], (num_movies, num_features), 'F')
    Theta = np.reshape(params[num_movies*num_features:], (num_features, num_users), 'F')
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)
    
    error = np.multiply(np.dot(X, Theta) - Y, R)
    X_grad = np.dot(error, Theta.T)
    Theta_grad = np.dot(X.T, error)
    grad = np.hstack((np.ravel(X_grad, order="F"), np.ravel(Theta_grad,order="F")))
    return grad
    

In [6]:
def computeNumericalGradientSinReg(X,Theta, Y, R, num_features):
    mygrad = np.zeros(Theta.size + X.size)
    perturb = np.zeros(Theta.size + X.size)
    myeps = 0.0001
    params = np.concatenate((np.ravel(X, order='F'), np.ravel(Theta, order='F')))

    for i in range(np.size(Theta)+np.size(X)):
        # Set perturbation vector
        perturb[i] = myeps
        params_plus = params + perturb
        params_minus = params - perturb
        cost_high = cofiCostFuncSinReg(params_plus, Y, R, num_features)
        cost_low = cofiCostFuncSinReg(params_minus, Y, R, num_features)

        # Compute Numerical Gradient
        mygrad[i] = (cost_high - cost_low) / float(2 * myeps)
        perturb[i] = 0

    return mygrad
    
def checkNNGradientsSinReg():
    #Create small problem
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    #Zap out most entries
    Y = X_t @ Theta_t.T
    dim = Y.shape
    aux = np.random.rand(*dim)
    Y[aux > 0.5] = 0
    R = np.zeros((Y.shape))
    R[Y != 0] = 1

    #Run Gradient Checking
    dim_X_t = X_t.shape
    dim_Theta_t = Theta_t.shape
    X = np.random.randn(*dim_X_t)
    Theta = np.random.randn(*dim_Theta_t)
    num_users = Y.shape[1]
    num_movies = Y.shape[0]
    num_features = Theta_t.shape[1]

    params = np.concatenate((np.ravel(X,order='F'), np.ravel(Theta,order='F')))

    # Calculo gradiente mediante aproximación numérica
    mygrad = computeNumericalGradientSinReg(X, Theta, Y, R, num_features)

    #Calculo gradiente
    grad = cofiGradientFuncSinReg(params, Y, R, num_features)

    # Visually examine the two gradient computations.  The two columns
    # you get should be very similar.
    df = pd.DataFrame(mygrad,grad)
    print(df)

    # Evaluate the norm of the difference between two solutions.
    # If you have a correct implementation, and assuming you used EPSILON = 0.0001
    # in computeNumericalGradient.m, then diff below should be less than 1e-9
    diff = np.linalg.norm((mygrad-grad))/np.linalg.norm((mygrad+grad))

    print('If your gradient implementation is correct, then the differences will be small (less than 1e-9):' , diff)
    

In [7]:
grad = cofiGradientFuncSinReg(params, Y_sub, R_sub, features)
print("Gradient without regularization at loaded parameters: \n", grad)

checkNNGradientsSinReg()


Gradient without regularization at loaded parameters: 
 [  8.82173071  -0.912555    -1.15814916  -1.01861973  -0.80927332
   0.06240312  -1.61451267  -2.04902333  -1.80216476  -1.43178441
   5.28790198   1.38082478   1.75244347   1.54131572   1.22454499
 -15.05835243   8.17919126 -11.25116736  -6.45575963   2.46384767
  -7.35105818   0.           0.           0.           0.
   0.           0.        ]
                    0
 1.918736    1.918736
 1.840584    1.840584
 3.805914    3.805914
 5.802030    5.802030
 1.571635    1.571635
 1.716965    1.716965
 1.227065    1.227065
-5.105329   -5.105329
-13.305014 -13.305014
-10.058631 -10.058631
-7.698054   -7.698054
 2.827791    2.827791
 2.215795    2.215795
 1.626214    1.626214
 18.804743  18.804743
-1.753967   -1.753967
 3.936657    3.936657
 3.807179    3.807179
-0.725739   -0.725739
-1.889847   -1.889847
-5.039660   -5.039660
-3.822340   -3.822340
-2.043286   -2.043286
-18.453928 -18.453928
 0.693816    0.693816
-1.589197   -1.589197


#### EJ04.
Implementar la función coste y la función gradiente con regularización en cofiCostFuncReg y cofiGradientFuncReg respectivamente. Se debe incluir el parámetro lambda inicializado a 1.5. La función coste debe devolver un coste de 31.34 aproximadamente si usa las matrices de parámetros X y Theta almacenadas en el fichero “ex8_movieParams.mat” para los 4 primeros usuarios, 5 primeras películas y 3 primeros atributos. Usar la función auxiliar checkNNGradientsReg con el parámetro lambda inicializado a 1.5 para verificar que los gradientes están bien calculados.

##### Solución:

In [8]:
def cofiCostFuncReg(params, Y, R, num_features, lambda_param):
    num_movies, num_users = Y.shape
    
    # Desenrollar parámetros
    X = np.reshape(params[:num_movies*num_features], (num_movies, num_features), 'F')
    Theta = np.reshape(params[num_movies*num_features:], (num_features, num_users), 'F')
    J = 0
    
    error = np.multiply(np.dot(X, Theta) - Y, R)
    squared_error = np.power(error, 2)
    J = 1/2 * np.sum(squared_error)
    J += lambda_param/2 * (np.sum(np.power(Theta, 2)) + np.sum(np.power(X, 2)))
    return J
    
    

In [9]:
def cofiGradientFuncReg(params, Y, R, num_features, lambda_param):
    num_movies, num_users = Y.shape
    
    # Desenrollar parámetros
    X = np.reshape(params[:num_movies*num_features], (num_movies, num_features), 'F')
    Theta = np.reshape(params[num_movies*num_features:], (num_features, num_users), 'F')
    
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)
    
    error = np.multiply(np.dot(X, Theta) - Y, R)
    X_grad = np.dot(error, Theta.T)
    Theta_grad = np.dot(X.T, error)
    X_grad += lambda_param * X
    Theta_grad += lambda_param * Theta
    grad = np.hstack((np.ravel(X_grad, order="F"), np.ravel(Theta_grad,order="F")))
    return grad


In [10]:
def computeNumericalGradientReg(X,Theta, Y, R, num_features, lambda_param):
    mygrad = np.zeros(Theta.size + X.size)
    perturb = np.zeros(Theta.size + X.size)
    myeps = 0.0001
    params = np.concatenate((np.ravel(X, order='F'), np.ravel(Theta, order='F')))

    for i in range(np.size(Theta)+np.size(X)):
        # Set perturbation vector
        perturb[i] = myeps
        params_plus = params + perturb
        params_minus = params - perturb
        cost_high = cofiCostFuncReg(params_plus, Y, R, num_features, lambda_param)
        cost_low = cofiCostFuncReg(params_minus, Y, R, num_features, lambda_param)

        # Compute Numerical Gradient
        mygrad[i] = (cost_high - cost_low) / float(2 * myeps)
        perturb[i] = 0

    return mygrad
    
def checkNNGradientsReg(lambda_param):
    #Create small problem
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    #Zap out most entries
    Y = X_t @ Theta_t.T
    dim = Y.shape
    aux = np.random.rand(*dim)
    Y[aux > 0.5] = 0
    R = np.zeros((Y.shape))
    R[Y != 0] = 1

    #Run Gradient Checking
    dim_X_t = X_t.shape
    dim_Theta_t = Theta_t.shape
    X = np.random.randn(*dim_X_t)
    Theta = np.random.randn(*dim_Theta_t)
    num_users = Y.shape[1]
    num_movies = Y.shape[0]
    num_features = Theta_t.shape[1]

    params = np.concatenate((np.ravel(X,order='F'), np.ravel(Theta,order='F')))

    # Calculo gradiente mediante aproximación numérica
    mygrad = computeNumericalGradientReg(X, Theta, Y, R, num_features, lambda_param)

    #Calculo gradiente
    grad = cofiGradientFuncReg(params, Y, R, num_features, lambda_param)

    # Visually examine the two gradient computations.  The two columns
    # you get should be very similar.
    df = pd.DataFrame(mygrad,grad)
    print(df)

    # Evaluate the norm of the difference between two solutions.
    # If you have a correct implementation, and assuming you used EPSILON = 0.0001
    # in computeNumericalGradient.m, then diff below should be less than 1e-9
    diff = np.linalg.norm((mygrad-grad))/np.linalg.norm((mygrad+grad))

    print('If your gradient implementation is correct, then the differences will be small (less than 1e-9):' , diff)
    

In [11]:
# Evaluate cost function and gradient function, both with regularization
lambda_param = 1.5
J = cofiCostFuncReg(params, Y_sub, R_sub, features, lambda_param)
print("\n\nCost with regularization at loaded parameters: ", J, "(this value should be about 31.34)")

grad = cofiGradientFuncReg(params, Y_sub, R_sub, features, lambda_param)
print("Gradient with regularization at loaded parameters: \n", grad)
checkNNGradientsReg(lambda_param)




Cost with regularization at loaded parameters:  66.01417620916024 (this value should be about 31.34)
Gradient with regularization at loaded parameters: 
 [ 10.39475896   0.25872185  -0.19588587  -0.33819299   0.59703352
  -0.53794482  -2.19295154  -2.8708041   -3.00249242  -1.27264956
   7.07908115   2.16262146   1.62674891   2.56203765   1.76747442
 -14.630187     8.93671108 -11.89904221  -8.98215726   1.78187498
  -8.06926491   0.39440815   0.47619366   1.27006666  -0.43097597
  -0.17263041  -0.01759684]
                    0
 3.046383    3.046383
-0.274555   -0.274555
-4.461107   -4.461107
 1.909060    1.909060
 0.400711    0.400711
 4.736365    4.736365
-2.067596   -2.067596
 3.906130    3.906130
 0.017854    0.017854
-14.035468 -14.035468
-6.611435   -6.611435
-1.093100   -1.093100
-4.142287   -4.142287
 4.728508    4.728508
-10.687180 -10.687180
-6.727321   -6.727321
 0.754565    0.754565
-5.410766   -5.410766
-2.816300   -2.816300
-4.071770   -4.071770
 1.693548    1.693548
 4

#### EJ05.
Inicializar de forma random con valores pequeños tanto la matriz X como la matriz Theta para todo el conjunto de datos, utilize la función np.random.rand() indicando las dimensiones en los parámetros de entrada. A continuación, entrenar con regularización para obtener los parámetros óptimos X y Theta usando la función fmin_cg de la librería scipy.optimize con 200 iteraciones y lambda con valor 1.5.

##### Solución:

In [12]:
# Useful Values
movies = Y.shape[0]  # 1682
users = Y.shape[1]  # 943
features = 10
lambda_param = 1.5
maxiter = 200

# Inicialización de X y Theta
X = np.random.rand(movies, features) * (0.24)
Theta = np.random.rand(features, users) * (0.24)
params = np.hstack((np.ravel(X, order='F'), np.ravel(Theta, order='F')))# Desenrollar: primero X luego Theta
# Algoritmo de optimización
fmin_1 = opt.fmin_cg(maxiter=maxiter, f=cofiCostFuncReg, x0=params, fprime=cofiGradientFuncReg, args=(Y, R, features, lambda_param))
# Enrollar el resultado
X_fmin = np.reshape(fmin_1[:movies*features], (movies, features), 'F')
Theta_fmin = np.reshape(fmin_1[movies*features:], (features, users), 'F')


         Current function value: 33089.943665
         Iterations: 200
         Function evaluations: 306
         Gradient evaluations: 306


c:\Users\danit\anaconda3\envs\entornoIA2425\lib\site-packages\scipy\optimize\_optimize.py:1659: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


#### EJ06.
Después del entrenamiento, conseguir la matriz de predicciones. Además, imprimir por pantalla la recomendación de las 10 películas con mejores puntuaciones para el usuario 2. Deben ser películas que no estuviesen previamente puntuadas por dicho usuario, para ello use np.where() con la correspondiente condición.

##### Solución:

In [13]:
predictions = np.dot(X_fmin, Theta_fmin)
# Solo el usuario j
j = 2
res_user = np.zeros((movies, 1))
pred_userj = predictions[:,j] # Seleccionar el usuario j
# Para cada película: A las que tenían valor previo le ponemos un 0 y a las que hemos predicho el valor de su predicción

res_user = np.where(R[:, j] == 0, pred_userj, 0).reshape(-1, 1)


idx = np.argsort(res_user, axis=0)[::-1] # Ordenar por las predicciones de menor a mayor y coger sus índice. [::-1] significa que le damos la vuelta a la salida: es decir lo colocamos de mayor a menor

# Leer el fichero con los nombres de cada película
movie_idx = {}
f = open('movie_ids.txt',encoding = 'ISO-8859-1')
for line in f:
    tokens = line.split(' ')
    tokens[-1] = tokens[-1][:-1]
    movie_idx[int(tokens[0]) - 1] = ' '.join(tokens[1:])
print("Top 10 movie predictions:")
for i in range(10):
    j = int(idx[i])
    print('Predicted rating of {0} for movie {1}.'.format(str(float(res_user[j])), movie_idx[j]))


Top 10 movie predictions:
Predicted rating of 4.748763094203087 for movie Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996).
Predicted rating of 4.4807325877085455 for movie Ruby in Paradise (1993).
Predicted rating of 4.429671930113992 for movie GoodFellas (1990).
Predicted rating of 4.225071879096433 for movie Good, The Bad and The Ugly, The (1966).
Predicted rating of 4.159097312445302 for movie Once Upon a Time in the West (1969).
Predicted rating of 4.132062403319584 for movie Flirting With Disaster (1996).
Predicted rating of 4.097492256327822 for movie Fast, Cheap & Out of Control (1997).
Predicted rating of 4.070430019903004 for movie Paths of Glory (1957).
Predicted rating of 4.06838448755807 for movie Casino (1995).
Predicted rating of 4.068323734735017 for movie Shall We Dance? (1996).


C:\Users\danit\AppData\Local\Temp\ipykernel_2488\4150413384.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  j = int(idx[i])
C:\Users\danit\AppData\Local\Temp\ipykernel_2488\4150413384.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Predicted rating of {0} for movie {1}.'.format(str(float(res_user[j])), movie_idx[j]))
